In [1]:
import sys
import os
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append(str(Path.cwd().parent))
from config import PATH_KIOSK_USER_PATTERNS_FOLDER, PATH_KIOSK_USER_PATTERNS_REPO, PATH_DATA_SHARING_FOLDER, PATH_C_DEWEY_FOLDER

# Example usage
kupdat04_path = PATH_C_DEWEY_FOLDER /'DEWEY'
execfile(PATH_KIOSK_USER_PATTERNS_REPO / 'constants.py')

In [2]:
data_endpoint = 'https://app.deweydata.io/api/v1/external/data/fldr_rrijeyqynvmxbip4x'
import numpy as np
import deweydatapy as ddp
execfile(PATH_KIOSK_USER_PATTERNS_REPO / 'functions/extract_file_list.py')


In [3]:
files, meta = extract_file_list(bulk_api_pass_by_retail_store_visitors, data_endpoint)
files.to_csv(PATH_KIOSK_USER_PATTERNS_REPO / 'data' / f'kupdat09a_pass_by retail store visitors.csv')
files = pd.read_csv(PATH_KIOSK_USER_PATTERNS_REPO / 'data' / f'kupdat09a_pass_by retail store visitors.csv')
files.head()

 
Metadata summary ------------------------------------------------
Total number of files: 768
Total files size (MB): 187,626.78
Date aggregation: None
Date partition column: None
Data min available date: None
Data max available date: None
-----------------------------------------------------------------
 
Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 16
Total number of files: 768
Total files size (MB): 187,626.78
Average single file size (MB): 244.59
Date partition column: None
Expires at: 2025-07-18T22:46:06.888948+00:00
-----------------------------------------------------------------


,Unnamed: 0,index,page,link,partition_key,file_name,file_extension,file_size_bytes,modified_at,external_id,download_link
0,0,0,1,https://app.deweydata.io/api/v1/downloads/0198...,NaN,retail-store-visitors-jan_0_0_0.csv.gz,.csv.gz,262160736,2025-05-28T15:26:42.376000+00:00,dwll_01981a90-ff2c-7fa0-bd25-3a0820f06c1e,https://app.deweydata.io/api/v1/downloads/0198...
1,1,1,1,https://app.deweydata.io/api/v1/downloads/0198...,NaN,retail-store-visitors-jan_0_0_10.csv.gz,.csv.gz,262151489,2025-05-28T15:32:07.026000+00:00,dwll_01981a90-ff2c-7fa0-bd25-3a18d6a26597,https://app.deweydata.io/api/v1/downloads/0198...
2,2,2,1,https://app.deweydata.io/api/v1/downloads/0198...,NaN,retail-store-visitors-jan_0_0_11.csv.gz,.csv.gz,25820994,2025-05-28T15:32:39.287000+00:00,dwll_01981a90-ff2c-7fa0-bd25-3a2311a3ba19,https://app.deweydata.io/api/v1/downloads/0198...
3,3,3,1,https://app.deweydata.io/api/v1/downloads/0198...,NaN,retail-store-visitors-jan_0_0_1.csv.gz,.csv.gz,262197371,2025-05-28T15:27:14.039000+00:00,dwll_01981a90-ff2c-7fa0-bd25-3a37116b05c5,https://app.deweydata.io/api/v1/downloads/0198...
4,4,4,1,https://app.deweydata.io/api/v1/downloads/0198...,NaN,retail-store-visitors-jan_0_0_2.csv.gz,.csv.gz,262189421,2025-05-28T15:27:46.227000+00:00,dwll_01981a90-ff2c-7fa0-bd25-3a4000582c36,https://app.deweydata.io/api/v1/downloads/0198...


In [10]:
error_dict = {}

for p in [8,14]:
    dest_dir = fr'R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/{p:02d}/'
    os.makedirs(dest_dir, exist_ok=True)
    execfile(PATH_KIOSK_USER_PATTERNS_REPO / 'constants.py')
    key = f'{p:02d}'
    print(key)
    # Ensure 'partition_key' in files and 'date' in sampled are both datetime type
    split = files[files['page'] == p]

    error_list = []
    for idx, row in split.iterrows():
        try:
            ddp.download_files(pd.DataFrame([row]), dest_folder=dest_dir, skip_exists=True)
        except Exception as e:
            error_list.append(idx)
            print(f"Download failed for index {idx}: {e}")
    error_dict[key] = pd.Series(error_list, dtype='object')

08
File already exists: R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/08/retail-store-visitors-jan_3_5_11.csv.gz
Skipping...
File already exists: R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/08/retail-store-visitors-jan_3_5_1.csv.gz
Skipping...
File already exists: R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/08/retail-store-visitors-jan_3_5_2.csv.gz
Skipping...
File already exists: R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/08/retail-store-visitors-jan_3_5_3.csv.gz
Skipping...
File already exists: R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/08/retail-store-visitors-jan_3_5_4.csv.gz
Skipping...
File already exists: R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/08/retail-store-visitors-jan_3_5_5.csv.gz
Skipping...
File already exists: R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/08/retail-store-visitors-jan

# QC

In [11]:
err_list_all = pd.DataFrame()
execfile(PATH_KIOSK_USER_PATTERNS_REPO / 'functions' /'compare_file_sizes.py')
files = pd.read_csv(PATH_KIOSK_USER_PATTERNS_REPO / 'data' / f'kupdat09a_pass_by retail store visitors.csv')
for p in range(1, 17):
    dest_dir = fr'R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors/{p:02d}'
    result = compare_file_sizes(dest_dir, files)
    df_result = pd.DataFrame(result)
    df_result['page'] = f'{p:02d}'
    err_list_all = pd.concat([err_list_all, df_result], ignore_index=True)
    

In [8]:
err_list_all

,file_name,file_path,expected_size,actual_size,page
0,retail-store-visitors-jan_3_7_3.csv.gz,R:/Global Diabetes Research Center/Patel/pass_...,262192215,115343360,08
1,retail-store-visitors-jan_6_6_2.csv.gz,R:/Global Diabetes Research Center/Patel/pass_...,262189345,159842304,14
2,retail-store-visitors-jan_6_7_1.csv.gz,R:/Global Diabetes Research Center/Patel/pass_...,262169700,243269632,14


In [9]:
from shutil import move

for p in range(1, 17):
    error_dest_dir = fr'R:/Global Diabetes Research Center/Patel/pass_by/Retail Store Visitors - Error/{p:02d}'
    os.makedirs(error_dest_dir, exist_ok=True)

    # Filter err_list_all to only include rows where page == p
    err_list_year = err_list_all[err_list_all['page'] == f'{p:02d}']
    for file_path in err_list_year['file_path']:
        try:
            move(file_path, error_dest_dir)
            print(f"Moved {file_path}")
        except Exception as e:
            print(f"Failed to move {file_path}: {e}")